In [24]:
import itertools
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from scipy import stats
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.weightstats import *

In [5]:
#pd.read_csv("./mirror_mouses.txt",header=None,names=['proportions'])
data = np.array([49,58,75,110,112,132,151,276,281,362])

In [10]:
np.median(data)

122.0

In [9]:
stats.wilcoxon(data - 200.)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [13]:
x1 = [22,22,15,13,19,19,18,20,21,13,13,15]
x2 = [17,18,18,15,12,4,14,15,10]

In [14]:
stats.mannwhitneyu(x1,x2)

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [15]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)
    

In [19]:
df=pd.read_csv("./_fd3c19abe7bdcb720b46008823715fc7_challenger.txt", sep="\t")

In [21]:
permutation_test()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0
5,Jun18.83,22.2,0
6,Aug30.83,22.8,0
7,Nov28.83,21.1,0
8,Feb03.84,13.9,1
9,Apr06.84,17.2,1


In [25]:
permutation_test(df.Incident, df.Incident.mean())

1.0

In [34]:
df.Incident.as_matrix()

/Users/roman/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1])

In [35]:
np.random.seed(0)

def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

temp1_median_scores = map(np.mean, get_bootstrap_samples(df.loc[df.Incident==1,'Temperature'].as_matrix(), 1000))
temp0_median_scores = map(np.mean, get_bootstrap_samples(df.loc[df.Incident==0,'Temperature'].as_matrix(), 1000))

delta_median_scores = map(lambda x: x[1] - x[0], zip(temp0_median_scores, temp1_median_scores))
print "95% confidence interval for the difference between medians",  stat_intervals(delta_median_scores, 0.05)

95% confidence interval for the difference between medians [-8.06457589 -1.45040179]


/Users/roman/py2/lib/python2.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/Users/roman/py2/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [37]:
np.random.seed(0)
permutation_test(df.loc[df.Incident==0,'Temperature'], df.loc[df.Incident==1,'Temperature'].as_matrix(),10000)

/Users/roman/py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


0.007